In [5]:
import girderformindlogger
import girder_client as gc
import pandas as pd
import numpy as np
import simplejson

In [6]:
girder = gc.GirderClient(host="localhost", port=8080)

# Plan

### Basics 

- [x] create a new user (userA)
- [x] get userA's applets and make sure the list is empty
- [x] userA creates an applet with an activity-set-url (url TBD)
- [x] get userA's applets and make sure we have the applet they just created
- [x] create userB
- [x] userA invites userB to the applet
- [x] make sure userB has an invite
- [x] userB accepts their invite
- [x] list userB's applets
- [x] userA invites userC (by email) to the applet
- [x] check that the applet user table has userC pending
- [x] create userC with the same email
- [x] check that userC has an invite
- [x] accept userC's applet invite
- [x] check userC's applets
- [x] userA, B and C submit responses to each activity in the applet
- [x] user B removes the applet without removing data
- [x] user C removes the applet AND removes data
- [x] delete userB and userC
- [x] userA deactivites their applet
- [x] delete userA
- [x] refresh an applet
- [x] add a schedule to an applet
- [ ] add test for nested sub-activities

### Backend TODOs

- [ ] userA can see user's B & C's data
- [x] get responses for each user in the last 7 days

# Create a new user

In [7]:
def testCreateUser(email=None):
    randomUser = np.random.randint(1000000)
    firstName = 'test'
    lastName = 'user'
    # NOTE: girder makes login and email lowercase!!!
    login = 'testuser{}'.format(randomUser)
    if not email:
        email = 'testemail{}@testemail.com'.format(randomUser)
    password = 'password'
    createUser = girder.post('user', parameters=dict(firstName=firstName,
                                        lastName=lastName,
                                        login=login,
                                        email=email,
                                        password=password
                                       )
               )
    # assert 'authToken' in createUser.keys(), 'user has no token, {}'.format(createUser)
    assert createUser['email'] == email, 'email does not match, {} {}'.format(createUser['email'], email)
    assert createUser['firstName'] == firstName, 'firstName does not match'
    assert createUser['lastName'] == lastName, 'lastName does not match'
    assert createUser['login'] == login, 'login does not match, {} {}'.format(createUser['login'], login)
    assert createUser['admin'] == False, 'user is an admin'
    assert createUser['_modelType'] == 'user', 'model is not user'
    assert createUser['public'] == False, 'user is public!'

    return createUser

In [8]:
def authenticate(user):
    girder.authenticate(username=user['login'], password='password')

In [9]:
userA = testCreateUser()
userB = testCreateUser()

#authenticate(userA)

# Get applets when the user doesn't have any

In [10]:
def getAppletsNewUser(new_user):
    girder.authenticate(username=new_user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == 0, 'a new user should have an empty list of applets. we get {}'.format(appletList)
    return 1

In [11]:
getAppletsNewUser(userA)

1

# get Applet by ID

want to check recently added applet here

In [12]:
def getAppletById(user, ar):
    girder.authenticate(username=user['login'], password='password')
    res = girder.get('applet/{}'.format(ar['_id']))
    assert res['applet']['_id'].split('applet/')[1] == ar['_id'], 'applet ids are not the same'
    return 1

# Add an applet

activity set info is below:

In [13]:
activitySetUrl = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activity-sets/ema-hbn/ema-hbn_schema'

act1 = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNEvening/ema_evening_schema'
act2 = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNMorning/ema_morning_schema'

act1Item = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNEvening/items/good_bad_day'
act2Item = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNMorning/items/sleeping_aids'

In [28]:
nestedActivitySet = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activity-sets/pediatric-screener/pediatric-screener_schema'

nact1 = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-Parent/pediatric_screener_parent_schema'
nact2 = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-SelfReport/pediatric_screener_selfreport_schema'

nact1Item = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-Parent/items/fidgety'
nact2Item = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-SelfReport/items/having_less_fun'




In [15]:
def addApplet(new_user, activitySetUrl):
    girder.authenticate(username=new_user['login'], password='password')

    # TODO: create an activity-set that JUST for testing.
    # make sure it has all the weird qualities that can break


    # for now, lets do the mindlogger demo
    activitySetUrl = activitySetUrl
    randomAS = np.random.randint(1000000)
    ar = girder.post('applet', parameters=dict(activitySetUrl = activitySetUrl, name='testActivitySet{}'.format(randomAS)))

    assert ar['_id'], 'there is no ID!'
    assert ar['meta']['activitySet']['url'] == activitySetUrl, 'the URLS do not match! {} {}'.format(ar['meta']['activitySet']['url'], activitySetUrl)
    
    assert getAppletById(new_user, ar) == 1, 'something wrong with getAppletById'
    return ar

In [16]:
# userA_applets = addApplet(userA, activitySetUrl)
userA_applets = addApplet(userA, nestedActivitySet)

In [18]:
def getAppletsUser(user, n=1):
    girder.authenticate(username=user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == n, 'a new user should have {} applets. we get {}'.format(n, len(appletList))
    return 1

In [19]:
getAppletsUser(userA, 1)

1

### test the expanded applet with nested sub-activities

In [23]:
# TODO: specify the applet instead of using the first one.
def getExpandedApplet(userA):
    authenticate(userA)
    expandedApplet = girder.get('user/applets')[0]
    return expandedApplet

In [24]:
expApplet = getExpandedApplet(userA)

In [31]:
assert len(expApplet['items']) == 59, 'there should be a total of 59 items for the whole applet'

In [33]:
assert len(expApplet['activities']) == 8, 'there should be 8 activities for the whole applet, but some are sub-activities'

### Refresh an applet

In [35]:
def refreshApplet(userA, userA_applets):
    authenticate(userA)
    appletId = userA_applets['_id']
    refreshResp = girder.get('applet/{}?refreshCache=true'.format(appletId))
    # TODO: somehow check that the refresh actually worked?
    return refreshResp

In [36]:
refreshed = refreshApplet(userA, userA_applets)

In [37]:
refreshed['applet']['url']

'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activity-sets/pediatric-screener/pediatric-screener_schema'

### add a schedule

In [38]:
def addSchedule(userA, userA_applets):
    scheduleString = """{"type":2,"size":1,"fill":true,"minimumSize":0,"repeatCovers":true,"listTimes":false,"eventsOutside":false,"updateRows":false,"updateColumns":false,"around":1567321200000,"events":[{"data":{"title":"EMA: Morning","description":"","location":"","color":"#673AB7","forecolor":"#ffffff","calendar":"","busy":true,"icon":"","URI":"http://repronim.org/schema-standardization/activity-sets/mindlogger-demo/mindlogger-demo_schema","notifications":[{"start":"09:00","end":null,"random":false,"notifyIfIncomplete":false}],"useNotifications":true},"schedule":{}}]}
    """
    schedule = simplejson.loads(scheduleString)
    authenticate(userA)
    appletId = userA_applets['_id']
    putResp = girder.put('applet/{}/constraints/'.format(appletId), data=dict(schedule=simplejson.dumps(schedule)))
    assert putResp['applet']['schedule'] == schedule

In [39]:
addSchedule(userA, userA_applets)

# Invite userB to userA's applet

In [40]:
def inviteUserToApplet(userA, userA_applets, userB):
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    authenticate(userA)
    inviteResp = girder.post('group/{}/invitation'.format(groupId), {
        "email": userB['email']
    })
    assert inviteResp['public'] == False, 'invite is public!'
    assert len(inviteResp['access']['users']), 'ths user was not added'
    return inviteResp

In [41]:
inviteB = inviteUserToApplet(userA, userA_applets, userB)

## Get a group's pending invites and check for a user

In [42]:
def checkInvitesForUser(userA, userA_applets, userB):
    authenticate(userA)
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    pendingInvites = girder.get('group/{}/invitation'.format(groupId))
    value = False
    for invite in pendingInvites:
        if invite['email'] == userB['email']:
            value = True
            break
    assert value, "email not in list of pending invites, {}".format(pendingInvites)
    return 1

In [43]:
checkInvitesForUser(userA, userA_applets, userB)

1

### Accept an applet invite

In [44]:
def checkForInvite(userB, userA_applets):
    authenticate(userB)
    pendingInvitesForUser = girder.get('user/invites')
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    assert len(pendingInvitesForUser), "this user has no invites"
    assert pendingInvitesForUser[0]['_id'] == groupId, "this user doesn't have the invite you expect"
    return groupId

In [45]:
def acceptAppletInvite(userB, userA_applets):
    groupId = checkForInvite(userB, userA_applets)
    authenticate(userB)
    resp = girder.post('group/{}/member'.format(groupId))
    assert resp['_modelType'] == 'group', "something weird about response, {}".format(resp)
    return 1

In [46]:
acceptAppletInvite(userB, userA_applets)

1

get a list of applet users (manager)

In [47]:
def getUserTable(userA, userA_applets):
    authenticate(userA)
    appletUsers = girder.get('applet/{}/users'.format(userA_applets['_id']))
    return appletUsers

check that userB has an applet

In [48]:
getAppletsUser(userB, 1)

1

### Invite someone who does not have an account yet

In [49]:
userCemail = 'randomuserc{}@test.com'.format(np.random.randint(1000000))
inviteC = inviteUserToApplet(userA, userA_applets, {'email': userCemail})

In [50]:
inviteC

{'_accessLevel': 2,
 '_id': '5d8a4ddf646bfddd7363f820',
 '_modelType': 'group',
 'access': {'users': [{'flags': [],
    'id': '5d8a4dc8646bfddd7363f818',
    'level': 2}]},
 'created': '2019-09-24T17:09:51.784000+00:00',
 'description': '',
 'name': 'Default testActivitySet464922 (5d8a4ddf646bfddd7363f81f) Users',
 'openRegistration': False,
 'public': False,
 'requests': [],
 'updated': '2019-09-24T17:20:40.905512+00:00'}

In [51]:
def checkAppletUserTableForUser(userA, userA_applets, userCemail):
    ut = getUserTable(userA, userA_applets)
    assert len(list(filter(lambda x: x['email'] == userCemail, ut))) == 1
    return ut

In [52]:
checkAppletUserTableForUser(userA, userA_applets, userCemail)

[{'_id': '5d8a5068646bfddd7363fa38',
  'email': 'randomuserc135288@test.com',
  'groups': [{'_id': '5d8a4ddf646bfddd7363f820',
    'name': 'Default testActivitySet464922 (5d8a4ddf646bfddd7363f81f) Users',
    'role': 'user',
    'status': 'pending'}]},
 {'_id': '5d8a4dc8646bfddd7363f818',
  'email': 'testemail647576@testemail.com',
  'groups': [{'_id': '5d8a4ddf646bfddd7363f820',
    'name': 'Default testActivitySet464922 (5d8a4ddf646bfddd7363f81f) Users',
    'role': 'user',
    'status': 'active'}]},
 {'_id': '5d8a4dc8646bfddd7363f81b',
  'email': 'testemail927966@testemail.com',
  'groups': [{'_id': '5d8a4ddf646bfddd7363f820',
    'name': 'Default testActivitySet464922 (5d8a4ddf646bfddd7363f81f) Users',
    'role': 'user',
    'status': 'active'}]},
 {'_id': '5d8a4dc8646bfddd7363f818',
  'email': 'testemail647576@testemail.com',
  'groups': [{'_id': '5d8a4ddf646bfddd7363f821',
    'name': 'Default testActivitySet464922 (5d8a4ddf646bfddd7363f81f) Editors',
    'role': 'editor',
    '

create that person's account

In [53]:
userC = testCreateUser(userCemail)

and check if they have an invite

In [54]:
checkInvitesForUser(userA, userA_applets, userC)

1

In [55]:
checkForInvite(userC, userA_applets)

'5d8a4ddf646bfddd7363f820'

In [56]:
acceptAppletInvite(userC, userA_applets)

1

check that userC has applets

In [57]:
getAppletsUser(userC, 1)

1

# Post a response

In [58]:
def postResponse(userA, actURI, itemURI):
    authenticate(userA)
    expandedApplet = girder.get('user/applets')[0]
    appletId = expandedApplet['applet']['_id'].split('/')[1]
    a = expandedApplet['activities'][actURI]
    activityId = a['_id'].split('/')[1]
    # itemURI = itemURI
    response = {}
    response[itemURI] = np.random.randint(2)

    resp = girder.post('response/{}/{}'.format(appletId, activityId),
                       data={'metadata': simplejson.dumps(
                       {
                           'activity': activityId,
                           'applet': appletId,
                           'responses': response
                       }
                       )})
    assert resp['_id'], 'response is weird and does not have an id'
    assert 'activity' in resp['meta'].keys(), 'response does not have an activity'
    assert 'applet' in resp['meta'].keys(), 'response does not have an applet'
    assert 'responses' in resp['meta'].keys(), 'response does not have an response'
    return resp

In [59]:
# postResponse(userA, act1, act1Item)
# postResponse(userA, act2, act2Item)

# Note: posting the response under the top-level activity, not the sub-activity.

postResponse(userA, nact1, nact1Item)
postResponse(userA, nact2, nact2Item)

# postResponse(userB)
# postResponse(userC)

{'_id': '5d8a5074646bfddd7363fa40',
 'baseParentId': '5d8a4dc8646bfddd7363f818',
 'baseParentType': 'user',
 'created': '2019-09-24T17:20:52.970877+00:00',
 'creatorId': '5d8a4dc8646bfddd7363f818',
 'description': 'Self-report questionnaire (3) response on 2019-09-24 at 10:20:52 PDT',
 'folderId': '5d8a5074646bfddd7363fa3e',
 'lowerName': '2019-09-24-10-20-52-pdt (1)',
 'meta': {'activity': {'@id': '5d8a5056646bfddd7363f9b5',
   'name': 'Self-report questionnaire (3)',
   'url': 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-SelfReport/pediatric_screener_selfreport_schema'},
  'allTime': {'responses': {'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-SelfReport/items/having_less_fun': [{'count': 1,
      'value': 1}]},
   'schema:duration': 'PT0.00534S',
   'schema:endDate': '2019-09-24T17:20:52.975340+00:00',
   'schema:startDate': '2019-09-24T17:20:52.970000+00:00'},
  'apple

### get last 7 days of responses

In [63]:
authenticate(userA)
appletId = userA_applets['_id']
last7 = girder.get('response/last7Days/{}'.format(appletId))
last7

{'responses': {'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-Parent/items/fidgety': [{'date': '2019-09-24',
    'value': 0}],
  'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-SelfReport/items/having_less_fun': [{'date': '2019-09-24',
    'value': 1}]}}

### Remove an applet without removing data

In [64]:
def removeApplet(userB, userA_applets):
    authenticate(userB)
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    girder.delete('group/{}/member?delete=false'.format(groupId))
    assert getAppletsUser(userB, 0), "user still has the applet that should be deleted."

removeApplet(userB, userA_applets)

### Remove an applet and the data

In [65]:
def deleteApplet(userC, userA_applets):
    authenticate(userC)
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    girder.delete('group/{}/member?delete=true'.format(groupId))
    assert getAppletsUser(userC, 0), "user still has the applet that should be deleted."

deleteApplet(userC, userA_applets)

### Deactivate an applet as admin

In [66]:
def deactivateApplet(userA, userA_applets):
    authenticate(userA)
    girder.delete('applet/{}'.format(userA_applets['_id']))
    assert getAppletsUser(userA, 0), "user still has the applet that should be deleted."

deactivateApplet(userA, userA_applets)

# Delete a user

In [67]:
def testDeleteUser(new_user):

    girder.authenticate(username=new_user['login'], password='password')

    deleteUser = girder.delete('user/{}'.format(new_user['_id']))

    assert deleteUser['message'] == 'Deleted user {}.'.format(new_user['login']), "{} does not equal {}".format(deleteUser['message'], 'Deleted user {}'.format(new_user['login']))
    
    return 1

In [68]:
testDeleteUser(userA)
testDeleteUser(userB)
testDeleteUser(userC)

1